# Tie-breaking votes

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_latimes as lat
import glob
import os
import requests
from pandas import json_normalize

In [3]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
pd.set_option("max_colwidth", None)

### Get congressional voting data for House and Senate from ProPublica

In [4]:
key = "sg5Q11oWW9pqLtmn8gVS0nOS179RousHDpeTgZkA"

In [5]:
base_url = "https://api.propublica.org/congress/v1/senate/votes/recent.json?offset="

In [6]:
links = []

for offset in range(1, 200):
    links.append(dict(chamber="senate", url=base_url + str(offset)))

In [7]:
jsons = []
for l in links:
    r = requests.get(l["url"], headers={"X-API-KEY": key})
    d = r.json()
    l["datadict"] = d
    jsons.append(l)

In [8]:
def processdict(d):
    data = json_normalize(d["datadict"]["results"], record_path=["votes"])
    return data

In [9]:
alldata = pd.concat([processdict(j) for j in jsons], sort=False).reset_index()

In [10]:
alldata_slim = alldata[
    [
        "index",
        "congress",
        "chamber",
        "session",
        "roll_call",
        "source",
        "url",
        "vote_uri",
        "question",
        "question_text",
        "description",
        "vote_type",
        "date",
        "time",
        "result",
        "tie_breaker",
        "tie_breaker_vote",
    ]
]

In [11]:
alldata_slim = alldata_slim.drop_duplicates()

In [12]:
alldata_slim[
    alldata_slim["tie_breaker"] == "Vice President of the United States"
].head()

,index,congress,chamber,session,roll_call,source,url,vote_uri,question,question_text,description,vote_type,date,time,result,tie_breaker,tie_breaker_vote
16,16,117,Senate,1,161,https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00161.xml,https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=117&session=1&vote=00161,https://api.propublica.org/congress/v1/117/senate/sessions/1/votes/161.json,On the Motion to Discharge,On the Motion to Discharge PN79-6,"Colin Hackett Kahl, of California, to be Under Secretary of Defense for Policy",1/2,2021-04-21,17:58:00,Motion to Discharge Agreed to,Vice President of the United States,Yea
35,15,117,Senate,1,161,https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00161.xml,https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=117&session=1&vote=00161,https://api.propublica.org/congress/v1/117/senate/sessions/1/votes/161.json,On the Motion to Discharge,On the Motion to Discharge PN79-6,"Colin Hackett Kahl, of California, to be Under Secretary of Defense for Policy",1/2,2021-04-21,17:58:00,Motion to Discharge Agreed to,Vice President of the United States,Yea
54,14,117,Senate,1,161,https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00161.xml,https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=117&session=1&vote=00161,https://api.propublica.org/congress/v1/117/senate/sessions/1/votes/161.json,On the Motion to Discharge,On the Motion to Discharge PN79-6,"Colin Hackett Kahl, of California, to be Under Secretary of Defense for Policy",1/2,2021-04-21,17:58:00,Motion to Discharge Agreed to,Vice President of the United States,Yea
73,13,117,Senate,1,161,https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00161.xml,https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=117&session=1&vote=00161,https://api.propublica.org/congress/v1/117/senate/sessions/1/votes/161.json,On the Motion to Discharge,On the Motion to Discharge PN79-6,"Colin Hackett Kahl, of California, to be Under Secretary of Defense for Policy",1/2,2021-04-21,17:58:00,Motion to Discharge Agreed to,Vice President of the United States,Yea
92,12,117,Senate,1,161,https://www.senate.gov/legislative/LIS/roll_call_votes/vote1171/vote_117_1_00161.xml,https://www.senate.gov/legislative/LIS/roll_call_lists/roll_call_vote_cfm.cfm?congress=117&session=1&vote=00161,https://api.propublica.org/congress/v1/117/senate/sessions/1/votes/161.json,On the Motion to Discharge,On the Motion to Discharge PN79-6,"Colin Hackett Kahl, of California, to be Under Secretary of Defense for Policy",1/2,2021-04-21,17:58:00,Motion to Discharge Agreed to,Vice President of the United States,Yea


In [13]:
tie_breakers = (
    alldata_slim[alldata_slim["tie_breaker"] == "Vice President of the United States"]
    .groupby(["date", "description", "roll_call"])
    .agg({"index": "count"})
    .reset_index()
)

---

In [14]:
votes_url = "https://api.propublica.org/congress/v1/117/senate/sessions/1/votes/"

In [15]:
links_votes = []

for vote in tie_breakers.roll_call.unique():
    links_votes.append(dict(chamber="senate", url=votes_url + str(vote) + ".json"))